In [2]:
# INTRODUCING DATAFRAMES

import pandas as pd

names = ['United States', 'Australia', 'Japan', 'India', 'Russia', 'Morocco', 'Egypt']
dr =  [True, False, False, False, True, True, True]
cpc = [809, 731, 588, 18, 200, 70, 45]
my_dict = {'country': names, 'drives_right': dr, 'cars_per_cap': cpc}

cars = pd.DataFrame(my_dict)

In [5]:
cars.shape    # this is an attribute 
cars.describe()    # quick statistics for the df

,cars_per_cap
count,7.000000
mean,351.571429
std,345.595552
min,18.000000
25%,57.500000
50%,200.000000
75%,659.500000
max,809.000000


In [8]:
cars.columns
cars.values

array([['United States', True, 809],
       ['Australia', False, 731],
       ['Japan', False, 588],
       ['India', False, 18],
       ['Russia', True, 200],
       ['Morocco', True, 70],
       ['Egypt', True, 45]], dtype=object)